In [2]:
Threads.nthreads()

8

In [3]:
using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPSimulators, POMDPTools, POMDPPolicies, ParticleFilters, DiscreteValueIteration, Plots

In [4]:
include("temps/BasicPOMCP_2.jl") 

┌ Warning: POMDPs.@implemented is deprecated, use POMDPLinter.@implemented instead.
└ @ POMDPs ~/.julia/packages/POMDPs/XBTe5/src/deprecated.jl:10


Main.BasicPOMCP

In [5]:
using Statistics
using Plots
using CSV
using DataFrames
using JLD
using Statistics

In [8]:
function build_pomdp(component_id, replacement_cost, inspection_cost)
    num_trials = 5;
    num_budgets = 50;
    horizon = 100;

    # num_components x num_budgets x num_trials x horizon sized array for each logging element
    state_histories = Array{Tuple{Int64,Int64},3}(undef, num_budgets+1, num_trials, horizon);
    action_histories = Array{String,3}(undef, num_budgets+1, num_trials, horizon);
    observation_histories = Array{Tuple{Int64,Int64},3}(undef, num_budgets+1, num_trials, horizon);
    reward_histories = Array{Float64,3}(undef, num_budgets+1, num_trials, horizon);
    survive_data = Array{Float64,2}(undef, num_budgets+1, num_trials);

    # println("Running simulation for component = "*component_id)
    dynamics = []
    open("dynamics/dynamics_"*component_id*".csv") do f
        # println("hi")
        line = 0  
        while ! eof(f) 
            # read a new / next line for every iteration          
            s = readline(f)
            s = parse.(Float64, split(chop(s; head=1, tail=1), ','))
            push!(dynamics,s)
            line += 1
        end
    end

    budgets = 0:1000/num_budgets:1000

    for budget in 1:num_budgets+1
        buildingprob = QuickPOMDP(
                actions = ["do-nothing", "inspect", "maintain"], 
                obstype = Tuple{Int64, Int64},
                discount = 0.95,

                transition = function(s,a)
                    next_states_inspect = []
                    next_states_nothing = []
                    for i in 0:s[1]
                        push!(next_states_inspect, (i,s[2]+inspection_cost))
                        push!(next_states_nothing, (i,s[2]))
                    end
                    if a == "maintain"
                        return Deterministic((100, s[2] + replacement_cost))
                    elseif a == "inspect"
                        return SparseCat(next_states_inspect, dynamics[s[1]+1])
                    elseif a == "do-nothing"
                        return SparseCat(next_states_nothing, dynamics[s[1]+1])
                    end
                end,

                observation = function(s,a,sp)
                    if a == "inspect"
                        return Deterministic(sp)
                    elseif a == "maintain"
                        return Deterministic(sp)
                    elseif a == "do-nothing"
                        return Deterministic((101, sp[2]))
                    end
                end,

                reward = function(s,a,sp)
                    if sp[2] > budgets[budget]
                        return -100
                    else
                        if sp[1] == 0
                            return -15
                        elseif sp[1] > 0
                            return 3
                        end
                    end
                end,

                initialstate = Deterministic((100, 0)))
        solver_q = BasicPOMCP.POMCPSolver(c=10.0);
        planner_q = solve(solver_q, buildingprob);

        for trial in 1:num_trials
            survive = 0;
            actions_taken = Vector{String}()
            count = 1
            for (s, a, o, r) in stepthrough(buildingprob, planner_q, "s,a,o,r", max_steps = horizon)
                state_histories[budget, trial, count] = s
                action_histories[budget, trial, count] = a
                observation_histories[budget, trial, count] = o
                reward_histories[budget, trial, count] = r

                #push!(actions_taken,a)
                if r == 3
                    survive_data[budget,trial] += 1
                end
                count += 1
            end
        end
    end
    save("results/state_histories_"*component_id*".jld", "data", state_histories)
    save("results/action_histories_"*component_id*".jld", "data", action_histories)
    save("results/observation_histories_"*component_id*".jld", "data", observation_histories)
    save("results/reward_histories_"*component_id*".jld", "data", reward_histories)
    save("results/survive_data_"*component_id*".jld", "data", survive_data)
end

build_pomdp (generic function with 1 method)

In [9]:
function parallel_cpu!(x,y,z)
    Threads.@threads for i in 1:length(x)
        build_pomdp(x[i],y[i],z[i])
    end
    return nothing
end
df = DataFrames.DataFrame(CSV.File("gen_admin_building_data_scaled.csv"))
component_ids = df[!,"id"]
replacement_costs = df[!,"replacement_cost_scaled"]
inspection_costs = df[!,"inspection_cost_scaled"]
parallel_cpu!(component_ids,replacement_costs,inspection_costs)